In [1]:
import json
import pandas as pd
from unidecode import unidecode
import re

with open('data/subset_epita.json') as f:
    data = json.load(f)
    
def get_docs_from_id(id):
    docs = []
    for o in data:
        if o['document_type_id'] == id:
            docs.append(o)
    return docs

datas = get_docs_from_id(8)

def preprocess(word):
    lower = str.lower(word)
    normalized = unidecode(lower)
    punc = '''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~'''
    for ele in normalized:
        if ele in punc:
            normalized = normalized.replace(ele, "")
    return normalized

def flatten(l):
    return [item for sublist in l for item in sublist]

# Récupérer les mots par paragraphe

In [2]:
def abs(a, b):
    r = a - b
    if r < 0:
        return -r
    return r

def regrouper_mots_par_hauteur(doc, tolerance):
    mots_regroupes = []
    
    keys = list(doc.keys())
    keys.pop()
    keys.pop()
    for key in keys:
        page = {}
        lines = doc[key]
        for line in lines:
            for w in line['line']['words']:
                hauteur = w['top']
                found = False
                for h in page:
                    if abs(h, hauteur) <= tolerance:
                        found = True
                        page[h].append(w['text'])
                        break
                if found == True:
                    continue
                if hauteur not in page:
                    page[hauteur] = []
                page[hauteur].append(w['text'])
        mots_regroupes.append(page)
    return mots_regroupes

tolerance = 0.02
docs = []

for doc in datas:
    resultat = regrouper_mots_par_hauteur(doc, tolerance)
    docs.append(resultat)

# Afficher les groupes de mots
doc = docs[4]
for page in doc:
    print("@@@@@@@@@@@@@@@@")
    for hauteur, mots in page.items():
        print(mots)

@@@@@@@@@@@@@@@@
['Mme', 'Schneider', 'Stéphanie']
['ML', 'ab', 'Dr', 'Ph.', 'VINCENT', 'BIOPLUS', 'Pharmacien', 'biologiste', 'Enregistré', 'n°:', '773706437']
['9', 'rue', 'de', 'la', 'Faïencerie', '77130', 'MONTEREAU', 'Tél.', '01.64.32.18.61', 'Fax', '01.64.32.60.38']
['AU', 'LABORATOIRE']
['Mme', 'Schneider', 'Stéphanie', '56,', 'chemin', 'Virginie', 'Pasquier']
['77130', 'MAROLLES', 'SUR', 'SEINE']
['Prescripteur', ':', 'Dr', 'BOURIOT', 'MARINE', '73', 'RUE', 'DE', 'MONTREUIL', '75011', 'PARIS']
['Patient', ':', 'Mme', 'Schneider', 'Stéphanie', 'Né(e)', 'le', '21.07.1989', 'Née', 'Schneider']
['Dossier', 'N°', '9AFA051022', '-', '02', 'reçu', 'le', '20.02.2019', 'à', '08:43', 'Prélevé', 'le', '20.02.2019', 'à', '08:41', 'Edité', 'le', '06.03.2019', 'à', '17:47']
['Identifiant', "d'accès", 'à', 'vos', 'résultats', ':', 'P-001243621']
['MICROBIOLOGIE']
['Examens', 'ci-dessous', 'réalisés', 'le', '22.02.2019', ',', 'validés', 'le', '22.02.2019']
['PRELEVEMENT', 'URO-GENITAL']
['Orig

# Récupérer le top des unités trouvées dans les documents

In [3]:
with open("utils/results/bioloinc_units.json", 'r') as fichier_json:
    units = json.load(fichier_json)

In [4]:
units

{'%  réactivité (avec DTT)': ['%'],
 '%  réactivité (sans DTT)': ['%'],
 '% CD10': ['%'],
 '% CD16+CD56+': ['%'],
 '% CD19+': ['%'],
 '% CD3': ['%'],
 '% CD3+CD4+CD8': ['%'],
 '% CD3-CD16+CD56+': ['%'],
 '% CD3-CD19+': ['%'],
 '% CD4': ['%'],
 '% CD45': ['%'],
 '% CD5': ['%'],
 '% CD8': ['%'],
 "% d'hématies foetales": ['%', '%{RBCs}'],
 "% d'hémoglobine foetale": ["% d'Hb", '%{Hb}'],
 '% de réactivité IgG anti-HLA classe 1': ['%'],
 '% de réactivité IgG anti-HLA classe 2': ['%'],
 '1,25 dihydroxy vitamines D': ['pmol/L', 'pg/mL'],
 '1-hydroxymidazolam': ['ng/mL'],
 '1-méthylhistidine': ['µmol/L', 'umol/L'],
 '1-méthylhistidine/créatinine': ['µmol/mmol créat', 'umol/mmol{creat}'],
 '11-déoxycortisol': ['nmol/L', 'ng/mL'],
 '11-déoxycortisol J01 après métopirone': ['nmol/L'],
 '11-déoxycortisol avant métopirone': ['nmol/L'],
 '17-hydroxyprogestérone': ['nmol/L', 'ng/mL'],
 '17-hydroxyprogestérone salivaire': ['nmol/L'],
 '17-hydroxyprégnénolone': ['nmol/L', 'ng/mL'],
 '2-aminoadipate': 

In [5]:
# Récupérer la liste des marqueurs avec le nombre d'occurences
top_120_df = pd.read_csv('utils/results/top_120.csv')

# Garder que les marqueurs qui sont apparus au moins une fois
top_50_df = top_120_df.drop('Unnamed: 0', axis=1)[:50]
top_50_list = list(map(lambda x : preprocess(x), list(top_50_df['grandeur'])))


# Modifier le dictionnaire des unités pour garder que ceux qui apparaissent
all_units = set()
units_found = {}
for key in units :
    normalized_key = preprocess(key)
    unit_lower = []
    for u in units[key]:
        all_units.add(u.lower())
        unit_lower += u.lower()
        
    if normalized_key in top_50_list:
        units_found[normalized_key] = unit_lower

all_units.add('t/l')
all_units.add('g/100ml')
all_units.add('/mm3')
all_units.add('giga/l')
all_units.add('u3')
all_units.add('/mm3')
all_units.add('picog.')
all_units.add('meg/l')
all_units.add('meq/l')
all_units.add('neg/ml')
all_units.add('10%/')
all_units.add('ng/nl')
len(all_units)

154

# Récupérer les synonymes des marqueurs

In [6]:
synonymes = pd.read_excel("data/variable_biologique_bioLOINC.xlsx", sheet_name="CKD1")

In [7]:
synonymes

,Variable (abréviation),Variable,Synonymes et exemples,RegExp,Unité (cible),"{RegExp unité, facteur d'échelle}",Plage valeur plausibles (unité cible),LOINC,Unité trouvé,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,ALB,Albuminurie volumique,"Albuminurie, Albumine (urines), Albumine urina...","[aA]lbuminurie|([aA]lbumin.{1,10}[uU]rin[^ ]*)...",mg/L,"{mg([ \/]|( par ))L, 1};{( g)([ \/]|( par ))L,...",0-10000,NaN,NaN,microalbuminurie : autre variable ?,NaN,NaN,NaN,NaN
1,ALB24h,Albuminurie 24h,"Albuminurie, Albumine (urines), Albumine urina...","[aA]lbuminurie|([aA]lbumin.{1,10}[uU]rin[^ ]*)...",mg/24h,"{mg([ \/]|( par ))24 ?h,1};{( g)([ \/]|( par )...",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CRE,Créatininurie volumique,"Créatininurie, Créatinine (urines), Créatinine...","[cC]r.atininurie|([[cR]r.atinin.{0,8}[uU]ri[^ ...",g/L,"{( g)([ \/]|( par ))L,1};{mg([ \/]|( par ))L, ...",0-10,NaN,umol/L,NaN,NaN,NaN,NaN,NaN
3,CRE24h,Créatininurie 24h,"Créatininurie, Créatinine (urines), Créatinine...","[cC]r.atininurie|([[cR]r.atinin.{0,8}[uU]ri[^ ...",g/24h,"{( g)([ \/]|( par ))24 ?h,1};{mg([ \/]|( par )...",0-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PRO,Protéinurie volumique,"Protéinuries, Protéines (urines), Protéines ur...","[pP]rot.inurie|([pP]rot.in.{1,10}[uU]rin[^ ]*)",mg/L,"{mg([ \/]|( par ))L, 1} ;{g([ \/]|( par ))L,0....",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PRO24h,Protéinurie 24h,"Protéinuries, Protéines (urines), Protéines ur...","[pP]rot.inurie|([pP]rot.in.{1,10}[uU]rin[^ ]*)",mg/24h,"{mg([ \/]|( par ))24 ?h,1}; {g([ \/]|( par ))2...",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,RAC,Rapport albuminurie/créatininurie,"Rapport albuminurie/créatininurie, Rapport alb...","(RAC )|(ACR )|([rR]apport [micro]{0,5}?[aA]lbu...",mg/g,"{mg([ \/]|( par ))mmol,1};{g([ \/]|( par ))mol,1}",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,RPC,Rapport protéinurie/créatininurie,"Rapport protéines/créatinine urinaire, ...","(RPC )|([rR]apport [pP]rot.in.{1,10}[cC]r.atin...",mg/g,"{mg([ \/]|( par ))mmol,1};{g([ \/]|( par ))mol,1}",0-10000,NaN,NaN,utile?,NaN,NaN,NaN,NaN
8,DFG,Débit de filtration glomérulaire,"Débit de filtration glomérulaire, DFG, glomeru...","(DFG )|(GFR )|(CKD[- ]?EPI)|([dD].bit.{1,5}[fF...","mL/min/1,73m2",mL/mi?n/1.73m,0-300,NaN,NaN,filtrer les formules ?,"{FRP34, 01/01/2023, ALB, 34, 5245-45}",NaN,NaN,"Regarder si mention de CKD-EPI ou MDRD, Exclur..."
9,HEMO,Hémoglobine,"Hb, Hémoglobine (attention a ne pas capter l'h...","(H[bB] )|([hH].moglobine(?!.{0,4}glyc[^ ]*))",g/dL,g/dL ; g/100mL,0-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
regex_syn = synonymes.iloc[:]['RegExp'].tolist()

list_regex_dict = []
for regex_unit in synonymes['{RegExp unité, facteur d\'échelle}']:
    unit_dict_list = []
    # Ex: {mg([ \/]|( par ))L, 1};{( g)([ \/]|( par ))L,0.001}
    for r_split in regex_unit.split(';'):
        # r_split ex: '{mg([ \/]|( par ))L, 1}'
        unit_dict = {}
        r_split_clean = r_split.replace('{', '').replace('}', '').strip()
            
        r_splits = r_split_clean.split(',')
        # ex r_splits : ['mg([ \/]|( par ))L', '1']
        
        if len(r_splits) > 1:
            unit_dict[r_splits[0]] = r_splits[1]
        else:
            unit_dict[r_splits[0]] = 1
        unit_dict_list.append(unit_dict)
    list_regex_dict.append(unit_dict_list)

synonymes['Regex_unité_facteur'] = list_regex_dict
synonymes

,Variable (abréviation),Variable,Synonymes et exemples,RegExp,Unité (cible),"{RegExp unité, facteur d'échelle}",Plage valeur plausibles (unité cible),LOINC,Unité trouvé,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Regex_unité_facteur
0,ALB,Albuminurie volumique,"Albuminurie, Albumine (urines), Albumine urina...","[aA]lbuminurie|([aA]lbumin.{1,10}[uU]rin[^ ]*)...",mg/L,"{mg([ \/]|( par ))L, 1};{( g)([ \/]|( par ))L,...",0-10000,NaN,NaN,microalbuminurie : autre variable ?,NaN,NaN,NaN,NaN,"[{'mg([ \/]|( par ))L': ' 1'}, {'( g)([ \/]|( ..."
1,ALB24h,Albuminurie 24h,"Albuminurie, Albumine (urines), Albumine urina...","[aA]lbuminurie|([aA]lbumin.{1,10}[uU]rin[^ ]*)...",mg/24h,"{mg([ \/]|( par ))24 ?h,1};{( g)([ \/]|( par )...",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'mg([ \/]|( par ))24 ?h': '1'}, {'( g)([ \/]..."
2,CRE,Créatininurie volumique,"Créatininurie, Créatinine (urines), Créatinine...","[cC]r.atininurie|([[cR]r.atinin.{0,8}[uU]ri[^ ...",g/L,"{( g)([ \/]|( par ))L,1};{mg([ \/]|( par ))L, ...",0-10,NaN,umol/L,NaN,NaN,NaN,NaN,NaN,"[{'( g)([ \/]|( par ))L': '1'}, {'mg([ \/]|( p..."
3,CRE24h,Créatininurie 24h,"Créatininurie, Créatinine (urines), Créatinine...","[cC]r.atininurie|([[cR]r.atinin.{0,8}[uU]ri[^ ...",g/24h,"{( g)([ \/]|( par ))24 ?h,1};{mg([ \/]|( par )...",0-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'( g)([ \/]|( par ))24 ?h': '1'}, {'mg([ \/]..."
4,PRO,Protéinurie volumique,"Protéinuries, Protéines (urines), Protéines ur...","[pP]rot.inurie|([pP]rot.in.{1,10}[uU]rin[^ ]*)",mg/L,"{mg([ \/]|( par ))L, 1} ;{g([ \/]|( par ))L,0....",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'mg([ \/]|( par ))L': ' 1'}, {'g([ \/]|( par..."
5,PRO24h,Protéinurie 24h,"Protéinuries, Protéines (urines), Protéines ur...","[pP]rot.inurie|([pP]rot.in.{1,10}[uU]rin[^ ]*)",mg/24h,"{mg([ \/]|( par ))24 ?h,1}; {g([ \/]|( par ))2...",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'mg([ \/]|( par ))24 ?h': '1'}, {'g([ \/]|( ..."
6,RAC,Rapport albuminurie/créatininurie,"Rapport albuminurie/créatininurie, Rapport alb...","(RAC )|(ACR )|([rR]apport [micro]{0,5}?[aA]lbu...",mg/g,"{mg([ \/]|( par ))mmol,1};{g([ \/]|( par ))mol,1}",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'mg([ \/]|( par ))mmol': '1'}, {'g([ \/]|( p..."
7,RPC,Rapport protéinurie/créatininurie,"Rapport protéines/créatinine urinaire, ...","(RPC )|([rR]apport [pP]rot.in.{1,10}[cC]r.atin...",mg/g,"{mg([ \/]|( par ))mmol,1};{g([ \/]|( par ))mol,1}",0-10000,NaN,NaN,utile?,NaN,NaN,NaN,NaN,"[{'mg([ \/]|( par ))mmol': '1'}, {'g([ \/]|( p..."
8,DFG,Débit de filtration glomérulaire,"Débit de filtration glomérulaire, DFG, glomeru...","(DFG )|(GFR )|(CKD[- ]?EPI)|([dD].bit.{1,5}[fF...","mL/min/1,73m2",mL/mi?n/1.73m,0-300,NaN,NaN,filtrer les formules ?,"{FRP34, 01/01/2023, ALB, 34, 5245-45}",NaN,NaN,"Regarder si mention de CKD-EPI ou MDRD, Exclur...",[{'mL/mi?n/1.73m': 1}]
9,HEMO,Hémoglobine,"Hb, Hémoglobine (attention a ne pas capter l'h...","(H[bB] )|([hH].moglobine(?!.{0,4}glyc[^ ]*))",g/dL,g/dL ; g/100mL,0-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'g/dL': 1}, {'g/100mL': 1}]"


# Itérer sur les paragraphes pour trouver les marqueurs

In [10]:
def search_marqueur(words, start):
    line = ' '.join(words[start:])
    marqueur = None
    regex = None
    for reg in regex_syn:
        marqueur = re.search(r'(?i)'+reg, line)
        if marqueur:
            marqueur = marqueur.group()
            regex = reg
            break
            
    if not marqueur :
        return -1, None, None
    
    marqueur_splits = marqueur.split()
    index_marqueur = -1
    for i in range(len(words[start:])):
        if marqueur_splits[-1] in words[start:][i]:
            index_marqueur = i
    return index_marqueur, marqueur, regex

pattern = re.compile('^[<>]?[0-9][.,]?[0-9]*')

def search_unit(words, start, marqueur):
    for i in range(start, len(words)):
        if words[i].lower() in all_units:
            return words[i].lower(), i
    return None, -1

def search_value(words, start, end):
    index = start
    while index < end and not re.search('^[<>]?[0-9][.,]?[0-9]*', words[index]):
        index+=1
        
    # Vérifier si la valeur est après l'unité
    if index == end:
        index+=1
        while index < len(words) and bool(pattern.match(words[index])) is True:
            index += 1
        return ' '.join(words[end + 1:index])
    return ' '.join(words[index:end])

In [11]:
def extract_values(doc):
    """
        doc : list of dict
    """
    values_found = {}
    for page in doc :
        for words in page.values():
            # Etape 1 : trouver le marqueur
            index_marqueur, marqueur, marqueur_regex = search_marqueur(words, 0)
            while index_marqueur != -1 :            
                # Etape 2 : trouver l'unité avec bioloinc
                unit_in_doc, index_unit = search_unit(words, index_marqueur, marqueur)
                if index_unit == -1:
                    break
                        
                # Etape 4 : récupérer la valeur qui est entre le marqueur et l'unité
                value = search_value(words, index_marqueur + 1, index_unit)
                if value == None:
                    break
                        
                """# Etape 5 : si l'unité est % alors on vérifie s'il y a le mot 'soit' ou ':' juste après
                if unit_in_doc == '%' and (words[index_unit + 1] == 'soit' or words[index_unit + 1] == ':'):
                    index_unit, unit_in_doc = search_unit(words, index_unit+2, marqueur)
                    value = search_value(words, index_marqueur, index_unit)
                    if value == None or index_unit == -1 :
                        break"""

                # Etape 6 : stocker dans un dictionnaire
                # Si le marqueur est dans les 50 marqueurs 
                #if marqueur in top_50_list :
                values_found[preprocess(marqueur).strip()] = value.replace(" ", "") + unit_in_doc
                
                # On relance pour vérifier s'il y a d'autres marqueurs sur la même ligne
                index_marqueur, marqueur, marqueur_regex = search_marqueur(words, index_unit + 1)

            
    return values_found

extract_values(docs[57])

<ipython-input-10-a23a2b10bbdd>:6: FutureWarning: Possible nested set at position 23
  marqueur = re.search(r'(?i)'+reg, line)


{'creatinine urinaire': '695mg/l', 'microalbuminurie': '3,3mg/l'}

In [43]:
for i in range(len(docs)) :
    print("Document index", i)
    print(extract_values(docs[i]))
    print('------------------')

Document index 0
{'leucocytes': '5000/ml'}
------------------
Document index 1
{}
------------------
Document index 2
{}
------------------
Document index 3
{'creatinine': '70,8umol/l', 'ckdepi': '75ans'}
------------------
Document index 4
{'hemoglobine': '13,5g/dl', 'leucocytes': '6400/mm3', 'lymphocytes': '39,4%', 'plaquettes': '264000/mm3', 'proteine c reactive': '<0,6mg/l', 'creatininemie': '8,3mg/l', 'creatinine': '89ml/min', 'ckdepi': '75ans', 'sodium': '136mmol/l', 'potassium': '4,4mmol/l', 'ferritine': '22ng/ml', 'asat': '20u/l', 'phosphatases alcalines': '43u/l', 'gammagt': '13u/l', 'tsh': '0,98mui/l', 'albumine': '55,50%'}
------------------
Document index 5
{'leucocytes': '5,1g/l', 'hemoglobine': '12,5g/dl', 'lymphocytes': '30,6%', 'plaquettes': '222g/l', 'ferritinex': '50ng/ml'}
------------------
Document index 6
{}
------------------
Document index 7
{}
------------------
Document index 8
{}
------------------
Document index 9
{'leucocytes': '4.000/ml'}
-----------------

KeyboardInterrupt: 